In [12]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import Markdown

import tb_incubator.targets as targets
from tb_incubator.constants import set_project_base_path
from tb_incubator.constants import compartments, latent_compartments, infectious_compartments, model_times, age_strata, indicator_names
from tb_incubator.model import build_model
from tb_incubator.utils import load_param_info, get_param_table
from tb_incubator.plotting import set_plot_label, display_plot

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")


## Model construction

In [13]:
param_info = load_param_info()
params = param_info["value"]

In [ ]:
model, desc = build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    params,
    model_times
)
Markdown(desc)

In [15]:
model.run(params)

## Parameters

In [ ]:
fixed_param_table = get_param_table(param_info)
Markdown(fixed_param_table.to_markdown())

## Results

In [ ]:
outs = model.get_derived_outputs_df()
fig = outs[[f"comp_size_{c}" for c in compartments]].plot.area()
set_plot_label(fig, indicator_names, "Compartment size")
#display_plot(fig, "comp_size", "svg")
fig

In [ ]:
fig = outs[["notification"]].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "notification", "svg")

In [ ]:
fig = outs["percentage_latent"].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "percentage_latent", "svg")

In [ ]:
fig = outs[["incidence", "prevalence", "mortality"]].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "inc_prev_mort", "svg")

In [ ]:
treatment_cov = targets.treatment_coverage
treatment_cov = pd.Series(treatment_cov)
fig = outs[["detection_rate"]].plot()
set_plot_label(fig, indicator_names, "value")
fig.add_trace(go.Scatter(x=treatment_cov.index, y=treatment_cov, mode='markers', name='Treatment coverage'))
display_plot(fig, "detection_rate", "svg")